In [13]:
import time
import pandas as pd
import os  # CSV 저장을 위한 폴더/파일 관리를 위해 import
# from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def clean_filename(filename):
    """파일 이름으로 사용할 수 없는 문자를 '_'로 대체합니다."""
    invalid_chars = '\\/*?:"<>|().'
    cleaned_name = filename.strip() # 양쪽 공백 제거
    for char in invalid_chars:
        cleaned_name = cleaned_name.replace(char, '_')
    cleaned_name = cleaned_name.replace(' ', '_') # 공백도 '_'로 대체
    # 혹시 모를 연속된 '_'를 하나로 변경 (선택적)
    while "__" in cleaned_name:
        cleaned_name = cleaned_name.replace("__", "_")
    return cleaned_name

def scrape_full_preview(driver, wait, scroll_wait):
    """
    [최종 수정됨] StaleElementReferenceException을 방지하기 위해
    루프 내에서 tbody를 계속 갱신하고, 파싱 직전에도 갱신합니다.
    """
    print("    [Sub] '데이터 미리보기' 탭의 그리드 로드 대기 중...")
    try:
        # 그리드 헤더와 본문이 로드될 때까지 대기
        wait.until(EC.visibility_of_element_located((By.ID, 'sheet1_Header')))
        # 초기 tbody 확인 (이 변수를 루프에서 재사용하지 않음)
        initial_tbody = wait.until(EC.visibility_of_element_located((By.ID, 'sheet1_Data')))
        print("    [Sub] 초기 데이터 로드 완료.")

        if "데이터 준비중입니다." in initial_tbody.text or "데이터가 없습니다." in initial_tbody.text:
            print("    [Sub] '데이터 준비중'이거나 데이터가 없어 이 항목을 스킵합니다.")
            return None

    except TimeoutException:
        print("    [Sub] 데이터 그리드(sheet1_Data)를 찾는 데 실패했습니다. (미리보기 미제공 추정)")
        return None

    # 1. 헤더(컬럼명) 추출
    header_elements = driver.find_elements(By.XPATH, "//thead[@id='sheet1_Header']//th")
    headers = [h.text for h in header_elements if h.text.strip() not in ['', 'No']]

    # 2. 모든 데이터 로드를 위한 스크롤 반복
    last_row_count = 0
    while True:
        try:
            # [수정] 스크롤 전에 매번 tbody를 새로 찾습니다.
            tbody = driver.find_element(By.ID, 'sheet1_Data')
            current_row_count = len(tbody.find_elements(By.TAG_NAME, 'tr'))

            # [수정] 이전 행 개수와 비교하여 더 이상 로드되지 않으면 탈출
            if current_row_count == last_row_count:
                print(f"    [Sub] 스크롤 완료. 총 {current_row_count}개 행 로드됨.")
                break

            last_row_count = current_row_count
            print(f"      [Sub] 스크롤... 현재 {current_row_count}개 행 로드됨.")

            # JavaScript를 실행하여 tbody의 스크롤을 맨 아래로 내림
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", tbody)

            # 스크롤 후 새 행이 로드될 때까지 (행 개수가 늘어날 때까지) 대기
            scroll_wait.until(
                lambda d: len(d.find_element(By.ID, 'sheet1_Data').find_elements(By.TAG_NAME, 'tr')) > current_row_count
            )

            time.sleep(0.5) # DOM 안정화를 위한 잠깐의 대기

        except TimeoutException:
            # 5초 동안 새 행이 로드되지 않으면(TimeoutException 발생)
            # 모든 데이터를 로드한 것으로 간주하고 루프 종료
            print(f"    [Sub] 스크롤 시간 초과(정상 종료). 총 {last_row_count}개 행 로드됨.")
            break
        except Exception as e:
            print(f"    [Sub] 스크롤 중 예외 발생: {e}")
            break # 스크롤 중 다른 오류 발생 시 중지

    # 3. 스크롤이 완료된 후, 전체 데이터 파싱
    print("    [Sub] 전체 데이터 파싱 시작...")
    all_data = []

    try:
        # [수정] 파싱 직전에도 tbody를 "반드시" 새로 찾습니다.
        final_tbody = driver.find_element(By.ID, 'sheet1_Data')
        data_rows = final_tbody.find_elements(By.TAG_NAME, 'tr')

        for row in data_rows:
            cells = row.find_elements(By.XPATH, './td[position()>1]')
            all_data.append([cell.text for cell in cells])

    except Exception as e:
        print(f"    [Sub] 최종 데이터 파싱 중 오류 발생: {e}")
        return None # 파싱 실패 시 None 반환

    # 4. Pandas DataFrame으로 변환
    df = pd.DataFrame(all_data, columns=headers)
    return df

# --- 메인 크롤링 로직 ---

# 크롤링할 URL
url = "https://www.bigdata-environment.kr/user/data_market/detail.do?id=1711a0a0-2f03-11ea-bccd-b704c648ae09"

# 제외할 파일 제목
EXCLUDED_TITLE = "행정동별 제과/아이스크림분야 소비인구"

# CSV 파일이 저장될 디렉토리 이름
OUTPUT_DIR = "scraped_data"

# Chrome WebDriver 설정 (자동 설치)
# service = Service(ChromeDriverManager().install())
# [수정] Chrome WebDriver 설정 부분
# 안정적인 실행을 위해 Chrome 옵션을 추가합니다.
chrome_options = Options()
chrome_options.add_argument("--no-sandbox") # 샌드박스 모드 비활성화 (권한 충돌 방지)
chrome_options.add_argument("--disable-dev-shm-usage") # 리소스 부족 문제 방지 (주로 Linux/Docker에서 필요하나 Windows에서도 도움됨)
chrome_options.add_argument("--disable-gpu") # GPU 가속 비활성화 (GPU 드라이버 충돌 방지)
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36") # 버전은 적당히 최신으로
# chrome_options.add_argument("--headless") # (선택사항) 아예 브라우저 창을 띄우지 않고 백그라운드에서 실행

print("Chrome 드라이버를 옵션과 함께 실행합니다...")

# [수정] 드라이버 설정 시 options=chrome_options 를 전달합니다.
# (이전 코드: driver = webdriver.Chrome())
# driver = webdriver.Chrome(options=chrome_options)
driver = uc.Chrome(options=chrome_options)

# 대기 시간 설정
wait = WebDriverWait(driver, 10) # 일반 대기 (10초)
scroll_wait = WebDriverWait(driver, 5) # 스크롤 후 새 데이터 로드 대기 (5초)

# 스크랩한 모든 데이터를 저장할 딕셔너리
all_scraped_data = {}

print(f"페이지 로드 중: {url}")
driver.get(url)

try:
    # '제공데이터' 목록이 있는 <ul> 요소를 찾습니다.
    data_list_container = wait.until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.data-provide-list > ul"))
    )

    # <li> 태그(각 파일 항목)의 총 개수를 셉니다.
    list_items = data_list_container.find_elements(By.TAG_NAME, "li")
    item_count = len(list_items)
    print(f"'제공데이터' 섹션에서 총 {item_count}개의 항목을 찾았습니다.")

    # 주의: 0부터 시작 (i = 0은 첫 번째 항목)
    for i in range(item_count):
        print(f"\n--- [Main] 항목 {i+1}/{item_count} 처리 중 ---")

        try:
            data_list_container = driver.find_element(By.CSS_SELECTOR, "div.data-provide-list > ul")
            current_item = data_list_container.find_elements(By.TAG_NAME, "li")[i]

            title_element = current_item.find_element(By.TAG_NAME, "dt")
            title = title_element.text.strip()
            print(f"  [Main] 제목: {title}")
        except Exception as e:
            print(f"  [Main] {i+1}번째 항목을 찾는 데 실패했습니다: {e}. 다음 항목으로 넘어갑니다.")
            continue

        # 1. 제외할 제목인지 확인
        if title == EXCLUDED_TITLE:
            print(f"  [Main] 제목이 '{EXCLUDED_TITLE}'와 일치하므로 스킵합니다.")
            continue

        # 2. 항목 클릭 (제목(dt)을 클릭하여 해당 파일 선택)
        try:
            print(f"  [Main] 항목 클릭: '{title}'")
            title_element.click()

            wait.until(
                EC.element_to_be_clickable((By.ID, "IBS_data_preview_Button"))
            )
            time.sleep(1) # AJAX 로딩을 위한 추가 대기

        except Exception as e:
            print(f"  [Main] 항목 클릭 중 오류 발생: {e}. 이 항목을 스킵합니다.")
            continue

        # 3. '데이터 미리보기' 탭 클릭
        try:
            print("  [Main] '데이터 미리보기' 탭 클릭...")
            preview_tab = driver.find_element(By.ID, "IBS_data_preview_Button")
            preview_tab.click()
        except Exception as e:
            print(f"  [Main] '데이터 미리보기' 탭 클릭 중 오류 발생: {e}. 이 항목을 스킵합니다.")
            continue

        # 4. 스크롤 및 스크래핑 함수 호출
        df = scrape_full_preview(driver, wait, scroll_wait)

        if df is not None:
            print(f"  [Main] '{title}' 항목의 데이터 {len(df)}행 스크랩 완료.")
            all_scraped_data[title] = df
        else:
            print(f"  [Main] '{title}' 항목에서 데이터를 스크랩하지 못했습니다 (미리보기 없음).")

    # --- 모든 루프 종료 ---
    print("\n=========================================")
    print("모든 크롤링 작업 완료.")

    if not all_scraped_data:
        print("스크랩된 데이터가 없습니다. (제외 항목만 있었거나 미리보기가 없는 항목뿐이었습니다.)")
    else:
        print(f"총 {len(all_scraped_data)}개의 파일에서 데이터를 스크랩했습니다.")

        # --- CSV 저장 로직 [추가된 부분] ---

        # 출력 디렉토리 생성 (없으면 만들기)
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        print(f"\n'{OUTPUT_DIR}' 폴더에 CSV 파일로 저장합니다...")

        for title, df in all_scraped_data.items():
            # 파일명으로 부적절한 문자 제거/변경
            safe_title = clean_filename(title)
            file_path = os.path.join(OUTPUT_DIR, f"{safe_title}.csv")

            try:
                # CSV 파일로 저장 (UTF-8 with BOM으로 Excel 호환성 확보)
                df.to_csv(file_path, index=False, encoding='utf-8-sig')
                print(f"  [저장 완료] '{title}' -> {file_path}")
            except Exception as e:
                print(f"  [저장 실패] '{title}' 저장 중 오류 발생: {e}")

        print("\n--- 저장 작업 완료 ---")
        # --- CSV 저장 로직 끝 ---

except Exception as e:
    print(f"전체 프로세스 중 심각한 오류가 발생했습니다: {e}")

finally:
    # 모든 작업이 끝나면 브라우저 종료
    print("브라우저를 닫습니다.")
    driver.quit()

ModuleNotFoundError: No module named 'undetected_chromedriver'

In [ ]:
import time
import json
import pandas as pd
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver

# 필수 모듈 임포트
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

def Bigdata_store():
    url = "https://www.bigdata-environment.kr/user/data_market/detail.do?id=1711a0a0-2f03-11ea-bccd-b704c648ae09"
    wd = webdriver.Chrome()
    wd.get(url)

    wait = WebDriverWait(wd, 10)

    try:
        # --- 1 ~ 3. (이전과 동일) ---
        # (전체보기 클릭 -> 팝업 -> li 클릭 -> 그리드 로드 확인)

        # --- (1. '전체보기' 버튼 클릭) ---
        xpath_selector = "//a[@onclick=\"modalPop('o','#pop_provide_data')\"]"
        view_all_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, xpath_selector))
        )
        print("XPath (onclick)로 '전체보기' 버튼을 찾았습니다. 클릭합니다.")
        wd.execute_script("arguments[0].click();", view_all_button)
        print("클릭 완료. 팝업 목록을 기다립니다...")

        # --- (2. 팝업 목록에서 li 클릭) ---
        # (첫 번째 li를 크롤링한다고 가정 : li:first-child)
        li_selector = "div#pop_provide_data div.provide-data-list ul li:nth-child(3)"
        li_element = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, li_selector))
        )
        li_text = li_element.text.replace('\n', ' ')[:40]
        print(f"팝업의 *세 번째* 항목 '{li_text}...'을(를) 클릭합니다.")
        li_element.click()
        print("*세 번째* li 클릭 완료. 데이터 미리보기 그리드를 기다립니다...")

        # --- (3. rMateH5 그리드 로드 대기) ---
        grid_cell_selector = "span.rMateH5__DataGridItemRenderer[id^='DataGridItemRenderer']"
        wait.until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, grid_cell_selector))
        )
        print("데이터 그리드 로드를 확인했습니다.")


        # --- 4. 가상 스크롤 처리 (인내심 로직 추가) ---

        scroll_container_selector = "div.rMateH5__VBrowserScrollBar"
        print(f"'{scroll_container_selector}' 컨테이너를 기준으로 스크롤을 시작합니다.")

        last_scroll_top = -1
        all_rows_list = []
        seen_rows_set = set()

        # [수정] "인내심" 카운터 추가
        patience_counter = 0

        # 헤더/푸터 필터링 목록
        HEADER_FOOTER_KEYWORDS = {
            "행정동명", "시도명", "시군구명","기준일자","성별","성별","연령대","소비인구(명)",
            "STD_DT", "sex_se", "year_se", "cnsmr_popltn_co",
        }

        while True:
            try:
                # (1. 파싱 로직은 동일)
                current_cells = wd.find_elements(By.CSS_SELECTOR, grid_cell_selector)
                current_row_data = []

                for cell in current_cells:
                    try:
                        aria_desc = cell.get_attribute("aria-describedby")

                        if aria_desc and aria_desc.endswith("_Column1"):
                            if current_row_data:
                                first_cell_data = str(current_row_data[0]).strip()
                                if first_cell_data not in HEADER_FOOTER_KEYWORDS:
                                    row_tuple = tuple(current_row_data)
                                    if row_tuple not in seen_rows_set:
                                        all_rows_list.append(current_row_data)
                                        seen_rows_set.add(row_tuple)
                                else:
                                    print(f"  (헤더/푸터 행 필터링됨: {first_cell_data}...)")
                            current_row_data = []

                        data = cell.get_attribute("title")
                        current_row_data.append(data)

                    except StaleElementReferenceException:
                        continue

                if current_row_data:
                    first_cell_data = str(current_row_data[0]).strip()
                    if first_cell_data not in HEADER_FOOTER_KEYWORDS:
                        row_tuple = tuple(current_row_data)
                        if row_tuple not in seen_rows_set:
                            all_rows_list.append(current_row_data)
                            seen_rows_set.add(row_tuple)
                    else:
                        print(f"  (헤더/푸터 행 필터링됨: {first_cell_data}...)")

                # (2. 스크롤 로직)
                scroll_container = wd.find_element(By.CSS_SELECTOR, scroll_container_selector)
                # [수정] 스크롤 속도 조절 (너무 빠르면 로드를 놓칠 수 있음)
                wd.execute_script("arguments[0].scrollTop += 800", scroll_container) # 1000 -> 800
                time.sleep(0.8) # 1.0 -> 0.8 (조절 가능)

                scroll_container = wd.find_element(By.CSS_SELECTOR, scroll_container_selector)
                current_scroll_top = wd.execute_script("return arguments[0].scrollTop", scroll_container)

                # (3. [핵심 수정] "인내심" 체크 로직)
                if current_scroll_top == last_scroll_top:
                    # 스크롤이 멈췄을 때
                    patience_counter += 1
                    print(f"  (스크롤 바닥 도달. 새 데이터 로드 대기... {patience_counter}/3)")

                    # 넉넉하게 3초 대기 (새 네트워크 요청 발생 대기)
                    time.sleep(3.0)

                    if patience_counter >= 10:
                        # 3번(총 9초)을 기다려도 스크롤 위치가 그대로라면,
                        # "진짜" 데이터의 끝으로 간주하고 종료
                        print("스크롤 완료. 모든 데이터가 로드되었습니다.")
                        break
                else:
                    # 스크롤이 성공했다면 (새 데이터가 로드되었거나, 아직 바닥이 아님)
                    # "인내심" 카운터 초기화
                    patience_counter = 0

                # (루프가 계속 돌 경우) 현재 위치 저장
                last_scroll_top = current_scroll_top
                print(f"새 데이터 로드 중... (현재 스크롤 위치: {current_scroll_top}) (수집된 행: {len(all_rows_list)})")

            except (StaleElementReferenceException, NoSuchElementException) as e:
                print("  (스크롤바 갱신 감지. 요소를 다시 찾습니다...)")
                time.sleep(0.5)
                continue

        # --- 5 & 6. (결과 출력 및 CSV 저장 로직 - 이전과 동일) ---
        print("\n중복 제거 및 헤더/푸터 필터링 완료.")

        print("\n--- 데이터 크롤링 완료 ---")

        if all_rows_list:
            print(f"총 {len(all_rows_list)}개의 고유한 *데이터 행*을 (순서대로) 추출했습니다.")

            df = pd.DataFrame(all_rows_list)
            try:
                # (첫 번째 li 데이터라고 가정)
                file_name = "icecream_sales_data(3)_2020/03-2020/10.csv"

                df.to_csv(file_name, index=False, encoding='utf-8-sig')

                print(f"\n[성공] 데이터가 {file_name} 파일로 저장되었습니다.")
                print(f" (Python 스크립트와 같은 폴더에 저장됨)")

            except PermissionError:
                print(f"\n[오류] CSV 파일 저장 실패: {file_name}이(가) 다른 프로그램에서 열려 있습니다.")
            except Exception as e:
                print(f"\n[오류] CSV 파일 저장 중 오류 발생: {e}")

            print(df.head())
            print(f"\n...")
            print(df.tail())
        else:
            print("추출된 데이터가 없습니다.")

    except Exception as e:
        print(f"전체 프로세스 중 오류 발생: {e}")

    finally:
        print("\n작업 완료. 브라우저를 닫습니다.")
        wd.quit()

# 함수 실행
if __name__ == "__main__":
    Bigdata_store()

XPath (onclick)로 '전체보기' 버튼을 찾았습니다. 클릭합니다.
클릭 완료. 팝업 목록을 기다립니다...
팝업의 *세 번째* 항목 '행정동별 제과/아이스크림분야 소비인구(2020.03~2020.05) 무료...'을(를) 클릭합니다.
*세 번째* li 클릭 완료. 데이터 미리보기 그리드를 기다립니다...
데이터 그리드 로드를 확인했습니다.
'div.rMateH5__VBrowserScrollBar' 컨테이너를 기준으로 스크롤을 시작합니다.
새 데이터 로드 중... (현재 스크롤 위치: 800) (수집된 행: 7)
  (헤더/푸터 행 필터링됨: STD_DT...)
  (헤더/푸터 행 필터링됨: sex_se...)
  (헤더/푸터 행 필터링됨: year_se...)
  (헤더/푸터 행 필터링됨: cnsmr_popltn_co...)
새 데이터 로드 중... (현재 스크롤 위치: 1600) (수집된 행: 29)
  (헤더/푸터 행 필터링됨: STD_DT...)
  (헤더/푸터 행 필터링됨: sex_se...)
  (헤더/푸터 행 필터링됨: year_se...)
  (헤더/푸터 행 필터링됨: cnsmr_popltn_co...)
새 데이터 로드 중... (현재 스크롤 위치: 2400) (수집된 행: 45)
  (헤더/푸터 행 필터링됨: STD_DT...)
  (헤더/푸터 행 필터링됨: sex_se...)
  (헤더/푸터 행 필터링됨: year_se...)
  (헤더/푸터 행 필터링됨: cnsmr_popltn_co...)
새 데이터 로드 중... (현재 스크롤 위치: 3200) (수집된 행: 61)
  (헤더/푸터 행 필터링됨: STD_DT...)
  (헤더/푸터 행 필터링됨: sex_se...)
  (헤더/푸터 행 필터링됨: year_se...)
  (헤더/푸터 행 필터링됨: cnsmr_popltn_co...)
새 데이터 로드 중... (현재 스크롤 위치: 4000) (수집된 행: 77)
  (헤더/푸터 행 필터링됨: STD_DT...)
  (헤더/푸터 행 필터링

In [ ]:
%%sql


In [ ]:
import requests
import pandas as pd
import time
import os

# --- 1. [필수] 사용자 입력: "Headers" 탭에서 복사 ---

# 1-1. 'Headers' 탭의 'General' 섹션에서 'Request URL'을 복사
API_URL = "https://www.bigdata-environment.kr/user/data_market/process.file.do" # (아마 이 주소일겁니다)

# 1-2. 'Headers' 탭의 'Request Headers' 섹션에서 값 복사
REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36', # (아까 찾은 User-Agent)
    'Referer': 'https://www.bigdata-environment.kr/user/data_market/detail.do?id=1711a0a0-2f03-11ea-bccd-b704c648ae09', # (아까 찾은 Referer)
    'Cookie': 'WMONID=QAJ_BPqziK9; _fwb=103CWL3q1nCaBgxSDeIHPra.1761099291921; JSESSIONID=142C4EB4882030A8257E5AD6E67B5BC3; _gid=GA1.2.1307590610.1761282715; _ga_CKPQWV25B6=GS2.1.s1761288815$o7$g1$t1761289620$j60$l0$h0; _ga=GA1.1.273076267.1761099292; wcs_bt=1e8330f8ae88d3:1761290957',
    'Origin': 'https://www.bigdata-environment.kr',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
}
# ---------------------------------------------------

# ★★★ [최종 완료] ★★★
# 다운로드할 3개 데이터셋 목록
DATASET_TO_DOWNLOAD = [
    {
        # [1번 데이터]
        "id": "7ffae3c0-8421-11ee-8bad-e375f0dc08ca",
        "filename": "data_1_2021_06_03.csv",
        "total_pages": 386 # (3,850,008건)
    }
    # {
    #     # [2번 데이터 (원래 3번째 항목)]
    #     "id": "20b9e320-308c-11eb-bc79-3b11eb915d6d",
    #     "filename": "data_2_2020_10_21_large.csv",
    #     "total_pages": 193 # (1,928,948건)
    # },
    # {
    #     # [3번 데이터 (원래 4번째 항목)]
    #     "id": "c846a500-e759-11ea-a837-83d4a69b8aa7",
    #     "filename": "data_3_2020_10_21_small.csv",
    #     "total_pages": 193 # (1,921,060건)
    # }
]

ROWS_PER_PAGE = 10000 # (Payload의 maxRows 값)
# ---------------------------------------------------


# 세션(Session)을 생성하여 연결 유지
session = requests.Session()
session.headers.update(REQUEST_HEADERS)

print(f"--- 총 {len(DATASET_TO_DOWNLOAD)}개의 데이터셋 다운로드를 시작합니다. ---")

# 3개의 데이터셋을 순서대로 처리
for dataset in DATASET_TO_DOWNLOAD:

    current_id = dataset["id"]
    current_filename = dataset["filename"]
    total_pages = dataset["total_pages"]

    print(f"\n\n--- 작업 시작: {current_filename} (ID: {current_id}) ---")
    print(f"총 {total_pages} 페이지, {ROWS_PER_PAGE * (total_pages - 1)}+ 건의 데이터를 수집합니다.")

    all_data_list = []

    if total_pages <= 0:
        print("[오류] total_pages가 0보다 커야 합니다. 이 데이터셋을 건너뜁니다.")
        continue

    try:
        # 1페이지부터 마지막 페이지까지 순차적으로 요청
        for page in range(1, total_pages + 1):
            payload = {
                'TP': 'one_sample',
                'id': current_id,
                'maxRows': ROWS_PER_PAGE,
                'currentPage': page
            }

            try:
                response = session.post(API_URL, data=payload)

                if response.status_code == 200:
                    data = response.json()
                    # (첫 스크린샷 'Preview' 탭에서 실제 데이터가 담긴 키 확인)
                    rows = data.get('preview_data') # <-- 이 키 이름이 맞는지 확인 필요

                    if rows:
                        all_data_list.extend(rows)
                        print(f"[성공] Page {page}/{total_pages} (데이터 {len(rows)}건 수집, 총 {len(all_data_list)}건)")
                    else:
                        print(f"[경고] Page {page}/{total_pages} - 응답 성공했으나 'preview_data' 키에 데이터 없음")
                else:
                    print(f"[실패] Page {page}/{total_pages} - 상태 코드: {response.status_code}")
                    print(f"  (응답 내용: {response.text[:100]}...)")
                    # 실패 시 몇 번 더 재시도하는 로직을 추가할 수 있습니다.

            except Exception as e:
                print(f"[오류] Page {page}/{total_pages} 요청 중 오류: {e}")
                break # 내부 오류 발생 시 현재 데이터셋 루프 중단

            # (권장) 서버에 부담을 주지 않도록 약간의 대기 시간
            time.sleep(0.5) # 0.5초 대기 (서버 상태에 따라 조절)

    except KeyboardInterrupt:
        print("\n사용자에 의해 중단. 현재 파일 저장을 시도합니다.")

    # --- 한 데이터셋 작업 완료 시 CSV 저장 ---
    if all_data_list:
        print(f"\n--- {current_filename} / 총 {len(all_data_list)}건 수집 완료 ---")
        print("CSV 파일로 저장 중...")

        try:
            df = pd.DataFrame(all_data_list)
            df.to_csv(current_filename, index=False, encoding='utf-8-sig')

            print(f"[최종 성공] '{current_filename}' 파일 저장 완료!")
            print(df.head())

        except Exception as e:
            print(f"[오류] CSV 파일 저장 중 오류 발생: {e}")

    else:
        print(f"\n[최종 실패] {current_filename} / 수집된 데이터가 없습니다.")

print("\n\n--- 모든 작업 완료 ---")

--- 총 1개의 데이터셋 다운로드를 시작합니다. ---


--- 작업 시작: data_1_2021_06_03.csv (ID: 7ffae3c0-8421-11ee-8bad-e375f0dc08ca) ---
총 386 페이지, 3850000+ 건의 데이터를 수집합니다.
[성공] Page 1/386 (데이터 10000건 수집, 총 10000건)
[성공] Page 2/386 (데이터 10000건 수집, 총 20000건)
[성공] Page 3/386 (데이터 10000건 수집, 총 30000건)
[성공] Page 4/386 (데이터 10000건 수집, 총 40000건)
[성공] Page 5/386 (데이터 10000건 수집, 총 50000건)
[성공] Page 6/386 (데이터 10000건 수집, 총 60000건)
[성공] Page 7/386 (데이터 10000건 수집, 총 70000건)
[성공] Page 8/386 (데이터 10000건 수집, 총 80000건)
[성공] Page 9/386 (데이터 10000건 수집, 총 90000건)
[성공] Page 10/386 (데이터 10000건 수집, 총 100000건)
[성공] Page 11/386 (데이터 10000건 수집, 총 110000건)
[성공] Page 12/386 (데이터 10000건 수집, 총 120000건)
[성공] Page 13/386 (데이터 10000건 수집, 총 130000건)
[성공] Page 14/386 (데이터 10000건 수집, 총 140000건)
[성공] Page 15/386 (데이터 10000건 수집, 총 150000건)
[성공] Page 16/386 (데이터 10000건 수집, 총 160000건)


In [1]:
import requests
import pandas as pd
import time
import os

# --- 1. [필수] 사용자 입력: "Headers" 탭에서 복사 ---

# 1-1. 'Headers' 탭의 'General' 섹션에서 'Request URL'을 복사
API_URL = "https://www.bigdata-environment.kr/user/data_market/process.file.do" # (아마 이 주소일겁니다)

# 1-2. 'Headers' 탭의 'Request Headers' 섹션에서 값 복사
REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36', # (아까 찾은 User-Agent)
    'Referer': 'https://www.bigdata-environment.kr/user/data_market/detail.do?id=1711a0a0-2f03-11ea-bccd-b704c648ae09', # (아까 찾은 Referer)
    'Cookie': 'WMONID=QAJ_BPqziK9; _fwb=103CWL3q1nCaBgxSDeIHPra.1761099291921; JSESSIONID=142C4EB4882030A8257E5AD6E67B5BC3; _gid=GA1.2.1307590610.1761282715; _ga_CKPQWV25B6=GS2.1.s1761288815$o7$g1$t1761289620$j60$l0$h0; _ga=GA1.1.273076267.1761099292; wcs_bt=1e8330f8ae88d3:1761290957',
    'Origin': 'https://www.bigdata-environment.kr',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
}
# ---------------------------------------------------

# ★★★ [최종 완료] ★★★
# 다운로드할 3개 데이터셋 목록
DATASET_TO_DOWNLOAD = [
    {
        # [1번 데이터]
        "id": "7ffae3c0-8421-11ee-8bad-e375f0dc08ca",
        "filename": "data_1_2021_06_03.csv",
        "total_pages": 386 # (3,850,008건)
    }
]

ROWS_PER_PAGE = 10000 # (Payload의 maxRows 값)
# ---------------------------------------------------


# 세션(Session)을 생성하여 연결 유지
session = requests.Session()
session.headers.update(REQUEST_HEADERS)

print(f"--- 총 {len(DATASET_TO_DOWNLOAD)}개의 데이터셋 다운로드를 시작합니다. ---")

# 3개의 데이터셋을 순서대로 처리
# (import os가 맨 위에 있는지 확인하세요)
import os

# 3개의 데이터셋을 순서대로 처리
for dataset in DATASET_TO_DOWNLOAD:

    current_id = dataset["id"]
    current_filename = dataset["filename"]
    total_pages = dataset["total_pages"]

    print(f"\n\n--- 작업 시작: {current_filename} (ID: {current_id}) ---")
    print(f"총 {total_pages} 페이지, {ROWS_PER_PAGE * (total_pages - 1)}+ 건의 데이터를 수집합니다.")
    print(f"데이터는 수집 즉시 '{current_filename}' 파일에 누적 저장됩니다.")

    # [수정] 전체 리스트 대신 총 수집 건수 카운터로 변경
    total_records_collected = 0

    # [수정] 이어받기 기능: 만약 파일이 이미 존재하면, 어디까지 받았는지 확인
    # (간단한 구현: 우선 기존 파일을 삭제하고 새로 시작합니다.)
    # (더 복잡한 이어받기를 원하시면 말씀해주세요)
    if os.path.exists(current_filename):
        print(f"[경고] '{current_filename}' 파일이 이미 존재합니다. 이어받기 대신, 기존 파일을 삭제하고 새로 시작합니다.")
        os.remove(current_filename)

    if total_pages <= 0:
        print("[오류] total_pages가 0보다 커야 합니다. 이 데이터셋을 건너뜁니다.")
        continue

    try:
        # 1페이지부터 마지막 페이지까지 순차적으로 요청
        for page in range(1, total_pages + 1):
            payload = {
                'TP': 'one_sample',
                'id': current_id,
                'maxRows': ROWS_PER_PAGE,
                'currentPage': page
            }

            try:
                response = session.post(API_URL, data=payload)

                if response.status_code == 200:
                    data = response.json()
                    rows = data.get('preview_data') # <-- 키 이름 확인 필요

                    if rows and len(rows) > 0:
                        # [핵심 수정] 데이터를 리스트에 담지 않고 DataFrame으로 바로 변환
                        df_chunk = pd.DataFrame(rows)

                        # [핵심 수정] 1페이지(파일이 없을 때)는 헤더와 함께 'w'(쓰기) 모드로,
                        # 2페이지부터는 헤더 없이 'a'(추가) 모드로 저장

                        # os.path.exists()로 확인하는 것이 'page == 1'보다
                        # 중단 후 재시작 시 더 안전합니다.
                        if not os.path.exists(current_filename):
                            print(f"  [저장] Page {page}: '{current_filename}'에 새 파일 생성 (헤더 포함).")
                            df_chunk.to_csv(current_filename, index=False, encoding='utf-8-sig', mode='w')
                        else:
                            print(f"  [저장] Page {page}: 기존 파일에 {len(rows)}건 추가.")
                            df_chunk.to_csv(current_filename, index=False, encoding='utf-8-sig', mode='a', header=False)

                        total_records_collected += len(rows)
                        print(f"[성공] Page {page}/{total_pages} (이번에 {len(rows)}건, 총 {total_records_collected}건 수집)")

                    else:
                        print(f"[경고] Page {page}/{total_pages} - 응답 성공했으나 데이터가 없음 (rows: {rows})")
                        # (데이터가 없는 페이지가 있다면 루프를 중단할 수도 있음)
                        # break
                else:
                    print(f"[실패] Page {page}/{total_pages} - 상태 코드: {response.status_code}")
                    print(f"  (응답 내용: {response.text[:100]}...)")
                    print("  [알림] 이 데이터셋의 수집을 중단하고 다음 데이터셋으로 넘어갑니다.")
                    break # 현재 데이터셋 작업 중단

            except Exception as e:
                print(f"[오류] Page {page}/{total_pages} 요청 또는 저장 중 오류: {e}")
                print("  [알림] 이 데이터셋의 수집을 중단하고 다음 데이터셋으로 넘어갑니다.")
                break # 현재 데이터셋 작업 중단

            # (권장) 서버에 부담을 주지 않도록 약간의 대기 시간
            time.sleep(1.0) # 0.5초 -> 1.0초 (안정성을 위해 1초로 늘림)

    except KeyboardInterrupt:
        print("\n사용자에 의해 중단. 현재까지 저장된 파일은 남아있습니다.")
        # 루프가 강제 중단되어도, 이미 저장된 CSV는 남아있음

    # --- 한 데이터셋 작업 완료 시 CSV 저장 (로직 변경) ---
    # [수정] 이미 저장이 완료되었으므로, 최종 요약만 출력
    if total_records_collected > 0:
        print(f"\n--- {current_filename} / 총 {total_records_collected}건 수집 완료 ---")
        print(f"[최종 성공] '{current_filename}' 파일 저장 완료!")

        # (파일이 너무 클 수 있으니 head() 대신 파일 존재 여부만 알림)
        # (미리보기를 원하면 아래 3줄의 주석을 해제하세요)
        # try:
        #     df_preview = pd.read_csv(current_filename, nrows=5)
        #     print(f"\n[파일 미리보기 (상위 5건)]\n{df_preview}")
        # except Exception as e:
        #     print(f"[알림] 파일 미리보기 중 오류: {e}")

    else:
        print(f"\n[최종 실패] {current_filename} / 수집된 데이터가 없습니다.")

print("\n\n--- 모든 작업 완료 ---")

--- 총 1개의 데이터셋 다운로드를 시작합니다. ---


--- 작업 시작: data_1_2021_06_03.csv (ID: 7ffae3c0-8421-11ee-8bad-e375f0dc08ca) ---
총 386 페이지, 3850000+ 건의 데이터를 수집합니다.
데이터는 수집 즉시 'data_1_2021_06_03.csv' 파일에 누적 저장됩니다.
  [저장] Page 1: 'data_1_2021_06_03.csv'에 새 파일 생성 (헤더 포함).
[성공] Page 1/386 (이번에 10000건, 총 10000건 수집)
  [저장] Page 2: 기존 파일에 10000건 추가.
[성공] Page 2/386 (이번에 10000건, 총 20000건 수집)
  [저장] Page 3: 기존 파일에 10000건 추가.
[성공] Page 3/386 (이번에 10000건, 총 30000건 수집)
  [저장] Page 4: 기존 파일에 10000건 추가.
[성공] Page 4/386 (이번에 10000건, 총 40000건 수집)
  [저장] Page 5: 기존 파일에 10000건 추가.
[성공] Page 5/386 (이번에 10000건, 총 50000건 수집)
  [저장] Page 6: 기존 파일에 10000건 추가.
[성공] Page 6/386 (이번에 10000건, 총 60000건 수집)
  [저장] Page 7: 기존 파일에 10000건 추가.
[성공] Page 7/386 (이번에 10000건, 총 70000건 수집)
  [저장] Page 8: 기존 파일에 10000건 추가.
[성공] Page 8/386 (이번에 10000건, 총 80000건 수집)
  [저장] Page 9: 기존 파일에 10000건 추가.
[성공] Page 9/386 (이번에 10000건, 총 90000건 수집)
  [저장] Page 10: 기존 파일에 10000건 추가.
[성공] Page 10/386 (이번에 10000건, 총 100000건 수집)
  [저장] Page 11: 기존 파일에 10